In [1]:
!mkdir -p /tmp/pip/cache/
!mkdir -p /root/.cache/torch/hub/checkpoints
!cp ../input/pawpularity-dataset-first-model/efficientnet_pytorch-0.7.1.xyz /tmp/pip/cache/efficientnet_pytorch-0.7.1.tar.gz
!cp ../input/pawpularity-dataset-first-model/efficientnet-b0-355c32eb.pth /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth
!pip install --no-index --find-links /tmp/pip/cache/ efficientnet_pytorch

Looking in links: /tmp/pip/cache/
Processing /tmp/pip/cache/efficientnet_pytorch-0.7.1.tar.gz
  Preparing metadata (setup.py) ... - done
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=d58d7f4400098c761a67983de21c9462100a2b91b5985d53a064c4dc4fe9a01f
  Stored in directory: /root/.cache/pip/wheels/02/50/84/82c97bfff69785108e55a6236d7ba11c83990eed25d4eab240
Successfully built efficientnet-pytorch


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
from efficientnet_pytorch import EfficientNet
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
from efficientnet_pytorch import EfficientNet
import torch.nn as nn
import torch.nn
import torchvision.transforms as T
from torchvision.io import read_image
from torch.utils.data import DataLoader, Dataset
import albumentations
import cv2
import os.path as p
from tqdm import tqdm
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        # print(os.path.join(dirname, filename))
        pass

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:

# import tez
# import timm
# import torchvision.models as models


class PetFinderModel(nn.Module):
    def __init__(self):
        super(PetFinderModel, self).__init__()
        self.eff_net = EfficientNet.from_name('efficientnet-b7')  # [m, 1000] for b1
        # self.bn1 = nn.BatchNorm1d(1000)
        self.drop_out = nn.Dropout(p=0.5)

        self.fc2 = nn.Linear(in_features=1000+12, out_features=64)
        # self.rl2 = nn.ReLU()
        self.fc3 = nn.Linear(in_features=64, out_features=1)

        # for p in self.eff_net.parameters():
        #     p.requires_grad = False

    def forward(self, x, meta):
        e = self.eff_net(x)  # [1, 1000] for b1
        # e_normed = self.bn1(e)
        # e_dropped = self.drop_out(e_normed)

        fr_r_meta = torch.cat((e, meta), dim=1)
        fr_r_meta_dropped = self.drop_out(fr_r_meta)

        f2 = self.fc2(fr_r_meta_dropped)  # FIXME
        # f2_r = self.rl2(f2)
        f2_r = torch.sigmoid(f2)
        # f2_rd = self.drop_out2(f2_r)

        f3 = self.fc3(f2_r)
        out = torch.sigmoid(f3) * 100

        return out

In [4]:
class PetFinderDataset(Dataset):
    def __init__(self, df, image_size=384, augment=False):
        self._X = df["Id"].values
        self._y = None
        self.augment = augment
        if "Pawpularity" in df.keys():
            self._y = torch.tensor(df["Pawpularity"].values, dtype=torch.float32)

        # handeling metadata
        b = df["Subject Focus"].values.reshape(-1, 1)
        c = df["Eyes"].values.reshape(-1, 1)
        d = df["Face"].values.reshape(-1, 1)
        e = df["Near"].values.reshape(-1, 1)
        f = df["Action"].values.reshape(-1, 1)
        g = df["Accessory"].values.reshape(-1, 1)
        h = df["Group"].values.reshape(-1, 1)
        i = df["Collage"].values.reshape(-1, 1)
        j = df["Human"].values.reshape(-1, 1)
        k = df["Occlusion"].values.reshape(-1, 1)
        l = df["Info"].values.reshape(-1, 1)
        m = df["Blur"].values.reshape(-1, 1)
        self.meta = torch.tensor(np.hstack((b, c, d, e, f, g, h, i, j, k, l, m)), dtype=torch.float32)
        if not self.augment:
            self._transform = T.Compose([
                # T.ToTensor(),
                # T.ConvertImageDtype(torch.float32),
                lambda x: x / 255,
                T.Resize([image_size, image_size]),
                # T.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                #                        std=[0.229, 0.224, 0.225])])
                T.transforms.Normalize(mean=[0.5, 0.5, 0.5],
                                       std=[0.5, 0.5, 0.5])
            ])

        # if not self.augment:
        #     self._transform = albumentations.Compose([
        #         albumentations.Resize(image_size, image_size, p=1),
        #         albumentations.Normalize(
        #             mean=[0.485, 0.456, 0.406],
        #             std=[0.229, 0.224, 0.225],
        #             max_pixel_value=255.0,
        #             p=1.0,
        #         ),
        #     ],
        #         p=1.0,
        #     )
        else:
            raise NotImplementedError

    def __len__(self):
        return len(self._X)

    def __getitem__(self, idx):
        image_path = self._X[idx]
        image = read_image(image_path).type(torch.float32)
        image = self._transform(image)
        #
        # my_img = image.type(torch.float32) / 255
        # my_img = (my_img - .5) / .5  # FIXME delete this

        # image = cv2.imread(self._X[idx])
        # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # image = self._transform(image=image)["image"]
        # image = np.transpose(image, (2, 0, 1)).astype(np.float32)  # permutation of dims

        meta = self.meta[idx]
        if self._y is not None:
            label = self._y[idx]
            return image, meta, label
        return image, meta



In [5]:
def predict(net, load_weights=True, pre_trained_params_path=None):
    dataset_path = "../input/petfinder-pawpularity-score"
    net.to("cuda:0")
    if load_weights:
        load_network_weights2(net, pre_trained_params_path)
    else:
        raise Exception("ERROR: need to load network weight")
    df = pd.read_csv(p.join(dataset_path, "test.csv"))
    id_temp = df["Id"].copy()
    df["Id"] = df["Id"].apply(lambda x: os.path.join(dataset_path, "test", x + ".jpg"))
    test_loader = torch.utils.data.DataLoader(PetFinderDataset(df), batch_size=32,
                                              num_workers=8, drop_last=False)
    test_iter = iter(test_loader)
    test_num_mini_batches = len(test_loader)
    predictions = np.array([])
    # net.eval()
    for _ in tqdm(range(test_num_mini_batches)):
        test_input, meta = test_iter.next()
        test_input, meta = test_input.to("cuda:0"), meta.to("cuda:0")
        test_output = net(test_input, meta)
        predictions = np.append(predictions, test_output.detach().cpu().numpy())

    df["Pawpularity"] = predictions
    df["Id"] = id_temp
    df = df[["Id", "Pawpularity"]]
    df.to_csv("submission.csv", index=False)
    print("finished predictions")
    return df

In [6]:
def predict2():
    super_final_predictions = []
    df_test = pd.read_csv("../input/petfinder-pawpularity-score/test.csv")
    id_temp = df_test["Id"].copy()
    df_test["Id"] = df_test["Id"].apply(lambda x: os.path.join("../input/petfinder-pawpularity-score/", "test", x + ".jpg"))
    
    net = PetFinderModel()
    net.to("cuda:0")
    load_network_weights2(net, "../input/cnn-v0113/CNN-v0.11.3_epoch_23_lowest16.979701434602642.pth")

    for fold in range(10):
        # model = torch_model(model_name="swin_large_patch4_window12_384")
        # model.load(f"../input/paw-models/model_f0.bin", device="cuda", weights_only=True)
        
#         net = PetFinderModel()
#         net.to("cuda:0")
#         load_network_weights2(net, "../input/cnnv059-epoch-19pth/CNN-v0.5.9_epoch_19.pth")

#         df_test = pd.read_csv("../input/petfinder-pawpularity-score/test.csv")
#         test_paths = [f"../input/petfinder-pawpularity-score/test/{x}.jpg" for x in df_test["Id"].values]
#         dense_features = ['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
#             'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur']

#         test = Dataset(
#             img_paths = test_paths,
#             dense_features = df_test[dense_features].values,
#             targets = np.ones(len(test_paths)),
#             augmentations=test_aug,
#         )
#         df_test = pd.read_csv("../input/petfinder-pawpularity-score/test.csv")
#         id_temp = df_test["Id"].copy()
#         df_test["Id"] = df_test["Id"].apply(lambda x: os.path.join("../input/petfinder-pawpularity-score/", "test", x + ".jpg"))
        
        # test_paths = [f"../input/petfinder-pawpularity-score/test/{x}.jpg" for x in df_test["Id"].values]
        
        test_loader = torch.utils.data.DataLoader(PetFinderDataset(df_test), batch_size=4,
                                                  num_workers=4, drop_last=False)
        test_iter = iter(test_loader)
        test_num_mini_batches = len(test_loader)
        test_predictions = np.array([])

        # test_predictions = model.predict(test, batch_size = Batch_size, n_jobs=-1)
        
        for _ in tqdm(range(test_num_mini_batches)):
            test_input, meta = test_iter.next()
            test_input, meta = test_input.to("cuda:0"), meta.to("cuda:0")
            test_output = net(test_input, meta)
            test_predictions = np.append(test_predictions, test_output.detach().cpu().numpy())

        final_test_predictions = []
        for preds in tqdm(test_predictions):
            final_test_predictions.extend(preds.ravel().tolist())

        # final_test_predictions = [sigmoid(x) * 100 for x in final_test_predictions]
        super_final_predictions.append(final_test_predictions)
        # del df_test

    super_final_predictions = np.mean(np.column_stack(super_final_predictions), axis=1)
    df_test["Pawpularity"] = super_final_predictions
    df_test["Id"] = id_temp
    df_test = df_test[["Id", "Pawpularity"]]
    df_test.to_csv("submission.csv", index=False)
    return df_test

In [7]:
def load_network_weights2(net, path):
    print("loading pre-trained weights from {}".format(path))
    net.load_state_dict(torch.load(path, map_location="cuda:0"))

In [8]:
# param_to_load = "../input/cnnv059-epoch-19pth/CNN-v0.5.9_epoch_19.pth"
# net = PetFinderModel() 
# df = predict(net, load_weights=True, pre_trained_params_path=param_to_load)
df_test = predict2()

loading pre-trained weights from ../input/cnn-v0113/CNN-v0.11.3_epoch_23_lowest16.979701434602642.pth


/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 8/8 [00:00<00:00, 50917.20it/s]


In [9]:
df_test.head()

,Id,Pawpularity
0,4128bae22183829d2b5fea10effdb0c3,28.829781
1,43a2262d7738e3d420d453815151079e,37.714205
2,4e429cead1848a298432a0acad014c9d,30.171717
3,80bc3ccafcc51b66303c2c263aa38486,25.578775
4,8f49844c382931444e68dffbe20228f4,33.510960
